### Redes Neuronales para Predicción de Precios de Acciones

Predecir la dirección del precio de una acción (subida/bajada) basado en:
- Nivel de cambio del precio de la posición
- Nivel de cambio del volumen transado

La red neuronal aprenderá patrones en estas variables para clasificar movimientos futuros del mercado.

In [45]:
import yfinance as yf

data = yf.download('BAP', period='1000d', interval='1d')
data['Price_Change'] = data['Close'].pct_change()
data['Volume_Change'] = data['Volume'].pct_change()

#data['Price_Change']


/tmp/ipykernel_810/2310341282.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BAP', period='1000d', interval='1d')
[*********************100%***********************]  1 of 1 completed


In [46]:
# el target refleja el movimiento del siguiente dia
# mientras para los feature es el mismo dia
data['Target'] = (data['Price_Change'].shift(-1) > 0).astype(int)

#la ultima fila quedará como NA pero dado que 
#NA > 0 return False entonces el valor del target en la
#última fila se llenará con 0
data = data.iloc[1:-1] # remuevo la primera y la última fila
#data['Target']

In [7]:
data[['Price_Change', 'Volume_Change', 'Target']].head(3)

Price,Price_Change,Volume_Change,Target
Ticker,,,
Date,,,
2025-02-12,-0.019425,-0.323040,0
2025-02-13,-0.005358,0.037895,1
2025-02-14,0.012679,-0.208587,0


In [27]:
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [47]:
X = data[['Price_Change', 'Volume_Change']].values
Y = data['Target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=.2, random_state=42)

x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32).view(-1, 1)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test, dtype=torch.float32).view(-1, 1)


In [ ]:
# para pytorch (m, n )
# m debe ser la cantidad de elementos
# n la cantidad de features
# (*, Hin) al final siempre se tiene el numero de feature de entrada
y_train.shape

torch.Size([70, 1])

In [48]:
class Net(nn.Module):
    def __init__(self, input_size, hidde_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidde_size = hidde_size
        self.fc1 = nn.Linear(input_size, hidde_size)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidde_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [49]:
model = Net(2, 15)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [52]:
epochs = 1000

for epoch in range(epochs):
    model.train()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 50 == 0:
        print(f'Epoch {epoch + 1}/ {epochs}, loss: {loss.item():.4f}')

Epoch 50/ 1000, loss: 0.6722
Epoch 100/ 1000, loss: 0.6701
Epoch 150/ 1000, loss: 0.6677
Epoch 200/ 1000, loss: 0.6653
Epoch 250/ 1000, loss: 0.6629
Epoch 300/ 1000, loss: 0.6612
Epoch 350/ 1000, loss: 0.6601
Epoch 400/ 1000, loss: 0.6594
Epoch 450/ 1000, loss: 0.6589
Epoch 500/ 1000, loss: 0.6584
Epoch 550/ 1000, loss: 0.6580
Epoch 600/ 1000, loss: 0.6574
Epoch 650/ 1000, loss: 0.6569
Epoch 700/ 1000, loss: 0.6563
Epoch 750/ 1000, loss: 0.6556
Epoch 800/ 1000, loss: 0.6549
Epoch 850/ 1000, loss: 0.6542
Epoch 900/ 1000, loss: 0.6535
Epoch 950/ 1000, loss: 0.6528
Epoch 1000/ 1000, loss: 0.6521


In [51]:

model.eval()

with torch.no_grad():
    y_pred_test = model(x_test)
    y_pred_labels = (y_pred_test > 0).float()
    accuracy = (y_pred_labels == y_test).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')
    

Test Accuracy: 0.4650


In [14]:
last_row = data[['Price_Change', 'Volume_Change']].iloc[-1].values
last_row_tensor = torch.FloatTensor(last_row).view(1, -1)

model.eval()

with torch.no_grad():
    predition = model(last_row_tensor)
    prediction_label = 1 if predition > .5 else 0

print(f'Prediction: {'price will increase ' if prediction_label == 1 
      else 'price will redice'}')





Prediction: price will increase 


Reset model weight

In [24]:
for name, param in model.named_parameters():
    print(f"{name} : {param.data.mean():.3f}")


fc1.weight : 0.088
fc1.bias : 0.066
fc2.weight : 0.120
fc2.bias : 0.360


In [42]:
def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_paramters'):
            layer.reset_parameters()

reset_weights(model)

print("model parameter after reset:")

for name, param in model.named_parameters():
    print(f"{name} : {param.data.mean():.4f}")




model parameter after reset:
fc1.weight : 0.1547
fc1.bias : -0.9573
fc2.weight : -0.2598
fc2.bias : 0.8145
